In [4]:
# Imports
import pandas as pd
import time
from googleapiclient.discovery import build
from config import api_key

In [5]:
# Read csv
df = pd.read_csv("large_channels.csv")
df.head()

,category,channel,video_url,video_id,fetch_status
0,Automobiles,https://www.youtube.com/@SupercarBlondie,https://www.youtube.com/watch?v=ZzWimhHches,ZzWimhHches,pending
1,Automobiles,https://www.youtube.com/@SupercarBlondie,https://www.youtube.com/watch?v=11m_OVKDHs4,11m_OVKDHs4,pending
2,Automobiles,https://www.youtube.com/@SupercarBlondie,https://www.youtube.com/watch?v=gVXJl_M1yfw,gVXJl_M1yfw,pending
3,Automobiles,https://www.youtube.com/@chrisfix,https://www.youtube.com/watch?v=PRSoRkM8GcM,PRSoRkM8GcM,pending
4,Automobiles,https://www.youtube.com/@chrisfix,https://www.youtube.com/watch?v=UEJbKLZ7RmM,UEJbKLZ7RmM,pending


# List of video ids
video_ids = df["video_id"].tolist()
video_ids[:10]

In [ ]:
# Counter
counter = 1

for id in video_ids:

    # empty list for storing reply
    replies = []

    # creating youtube resource object
    youtube = build('youtube', 'v3',
                    developerKey=api_key)

    # retrieve youtube video results
    video_response=youtube.commentThreads().list(
    part='snippet',
    videoId=id
    ).execute()


  # iterate video response
    while video_response:

        # extracting required info
        # from each result object
        for item in video_response['items']:

            # Extracting comments
            comment = item['snippet']['topLevelComment']['snippet']['textOriginal']

            # Store reply is list
            replies.append(comment)

            # print comment with list of reply
            print(comment, end = '\n\n')

        # Again repeat
        if 'nextPageToken' in video_response and len(replies)<1000:
            video_response = youtube.commentThreads().list(
                    part = 'snippet',
                    videoId = id,
                    pageToken = video_response['nextPageToken']
                ).execute()
        else:
            break

    df = pd.DataFrame(replies, columns=["Text"])

    # Drop duplicates from text column
    df = df.drop_duplicates(subset = ['Text'],keep='first')

    # Droping rows with href URL
    df = df[df["Text"].str.contains(r'href=\S+') == False]

    # Droping rows with non English words
    df=df[df.Text.map(lambda x: x.isascii())]

    df.to_csv(f"comments_{counter}.csv",index=False)
    counter += 1